# Twitter code

In [4]:
import pandas as pd
import tweepy
from tqdm.notebook import tqdm
data = pd.read_excel(r"Desktop\Book1.xlsx")#Here give location to your file containing handler name with a header in file

tmp = []
na = []

with tqdm(total=len(data), desc = 'Extracting') as pbar:
    for name in data.iloc[:,0]: 
    
        auth = tweepy.OAuthHandler('ENTER YOUR KEYS', 'ENTER YOUR KEYS') 
        auth.set_access_token('ENTER YOUR KEYS', 'ENTER YOUR KEYS') 
        api = tweepy.API(auth)

        alltweets = []

        new_tweets = api.user_timeline(screen_name = name,count=200)

        alltweets.extend(new_tweets)

        oldest = alltweets[-1].id - 1

        while len(new_tweets) > 0:

            new_tweets = api.user_timeline(screen_name = name,count=200,max_id=oldest)

            alltweets.extend(new_tweets)

            oldest = alltweets[-1].id - 1

        outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]
        for j in outtweets: 
            tmp.append(j)
            na.append(name)
        pbar.update(1)
    
df = pd.DataFrame({'Tweets':tmp, 'Handler':na})
df.to_excel(r"Desktop\Twitter Output.xlsx") # excel file will be export on desktop

# Rating table scraping code Amazon

In [6]:
from time import sleep
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from selenium import webdriver

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

data = pd.read_excel(r'Desktop\Book2.xlsx')#Here give location to your file containing seller url's name with a header in file


dfs=[]
with tqdm(total=len(data), desc = 'Scraping') as pbar:
    with webdriver.Chrome(r"C:\chromedriver_win32\chromedriver.exe") as driver:
            for url in data.iloc[:,0]:
                driver.get(url)
                sleep(3)
                soup = BeautifulSoup(driver.page_source)

                d = []
                positive=[]
                neutral = []
                negative=[]
                count = []

                for i in soup.findAll('td', class_ = 'a-text-right'):
                    d.append(i.text.strip())
                for i in d[:4]:
                    positive.append(i)
                for i in d[4:8]:
                    neutral.append(i)
                for i in d[8:12]:
                    negative.append(i)
                for i in d[12:]:
                    count.append(i)

                e = pd.DataFrame({'Positive':positive, 'Neutral':neutral, 'Negative':negative, 'Count':count})
                e = e.T
                df = e.set_axis(['30 days', '90 days', '12 months', 'Lifetime'], axis=1, inplace=False)
                df['Seller URL'] = driver.current_url
                dfs.append(df)
                pbar.update(1)
df_final = dfs[0]
for i in range(len(dfs)-1):
    df_final = pd.concat([df_final, dfs[i+1]])
df_final
df_final.to_excel(r'Desktop\Amazon Output.xlsx')